In [14]:
import tensorflow as tf
import os
import cv2
import imghdr
import numpy as np
import matplotlib.pyplot as plt

In [5]:
data_dir = 'datasets'

In [6]:
images_exts = ['jpeg', 'jpg', 'bmp', 'png']

In [10]:
for image_class in os.listdir(data_dir):
    print(image_class)
    for image in os.listdir(os.path.join(data_dir , image_class)):
        image_path = os.path.join(data_dir , image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in images_exts:
                print(f"{image} doesnot exits in the extension")
                os.remove(image_path)
        except Exception as e:
            print(f"Issue with the image {image}")


Beagle
Beagle-Hero.jpg doesnot exits in the extension
beagle-on-meadow-2021-08-26-15-58-22-utc.jpg doesnot exits in the extension
blog-breed-beagle_1.width-550.format-webp.webp doesnot exits in the extension
GettyImages-157603001-e1701106766955.jpg doesnot exits in the extension
vector.svg doesnot exits in the extension
vector10.svg doesnot exits in the extension
vector11.svg doesnot exits in the extension
vector12.svg doesnot exits in the extension
vector13.svg doesnot exits in the extension
vector2.svg doesnot exits in the extension
vector3.svg doesnot exits in the extension
vector4.svg doesnot exits in the extension
vector5.svg doesnot exits in the extension
vector6.svg doesnot exits in the extension
vector7.svg doesnot exits in the extension
vector8.svg doesnot exits in the extension
vector9.svg doesnot exits in the extension
German_Shepherd
0_GettyImages-1477090988.jpg doesnot exits in the extension
220px-German_Shepherd_-_DSC_0346_281009636283329.jpg doesnot exits in the extensio

In [11]:
for image_class in os.listdir(data_dir):
    print(image_class)

Beagle
German_Shepherd
Golden_Retriever
Labradorr_retriever
Siberian_Husky


In [12]:
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir , image_class)):
        print(image)

00xp-beagles-AP-facebookJumbo-v2.jpg
2-Jahre-Beagle-768x512.jpg
4195-wMr6JS._SL500_.jpg
47e924f8d028974fa04275be6b8f185f.jpg
61.-Beagle.jpg
640px-Beagle_Upsy.jpg
67e6a3f1bf14516c30d2cdad0042e993.jpg
7dec9387a32071dbc7a3bcffdc08de89Y29udGVudHNlYXJjaGFwaSwxNjkyODkzNDA2-2.73426158.jpg
800px-Beagle_harrier.JPG.jpg
a8b50632a4f324c88fa11e310b027b1d.png
adorable-beagle-puppy-solo-portrait_53876-73998.jpg
Are-Beagles-Good-Dogs.jpg
Basset-Hound-vs-Beagle-1-.jpg
beagle-1.jpg
beagle-12.jpg
beagle-13.jpg
Beagle-2-1-1024x768.jpeg
beagle-2_custom-01105a795e7709dcb99ee953d438ef5ca4781622.jpg
beagle-5375285_1280.jpg
Beagle-breed-guide-1.jpg
beagle-card-large.jpg
Beagle-Dog-Breed-Image-1024x722.jpg
beagle-dog-breed-of-the-month-1619614746.jpg
beagle-dog-sitting-with-white-background_53876-30186.jpg
beagle-dog.jpg
Beagle-e1587511922360.jpg
Beagle-head-portrait-in-a-field-in-early-morning-500x486.jpg
beagle-head.png
beagle-hero-image-for-breed-page.jpg
Beagle-hound-dog-puppy.jpg
beagle-hound-dog.jpg
beag